<a href="https://colab.research.google.com/github/sebastian2161/master_rating/blob/main/master_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Despliegue en google colab - google cloud conectado a backend de Google Compute Engine
#Utilice este ambiente porque es para testing, demos o pruebas, gratuito que ofrece google cloud
#Tiene las funciones de un jupyter notebook, entrega los resultados por parte o trozos del codigo 

#Crea la carpeta data de forma automatica parte 7 , ruta /content/data, con los archivos.csv
#Crea la carpeta output de forma automatica parte final, ruta /content/ouput, con el archivo maestro_rating.csv

#Instalacion open jdk
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
#Instalacion de Apache Hadoop

import os # libreria de manejo del sistema operativo
os.system("wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz")
os.system("tar xf /spark-2.4.5-bin-hadoop2.7.tgz")

512

In [4]:
#Instalacion de PySpark

!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.5 MB/s eta 0:00:00


In [5]:
# Cargar Pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()
spark

In [6]:
# Exportar el archivo data.xlsx con todas sus hojas

from google.colab import files
files.upload()

Saving data.xlsx to data.xlsx


{'data.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00Js\xd2\xd8m\x01\x00\x00(\x06\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [38]:
import pandas as pd
#Separar la hoja rating_empresa y convertirlo en un dataframe pandas
hoja_rating_empresa= pd.read_excel('data.xlsx',sheet_name='rating_empresa')

#Separar la hoja rating_soberano y convertirlo en un dataframe pandas
hoja_rating_soberano= pd.read_excel('data.xlsx',sheet_name='rating_soberano')

#Eliminar el caracter 'u' en los campos sp, mdy,fitch para el rating soberano
hoja_rating_soberano['sp'] = hoja_rating_soberano['sp'].replace({'u':''}, regex=True)
hoja_rating_soberano['mdy'] = hoja_rating_soberano['mdy'].replace({'u':''}, regex=True)
hoja_rating_soberano['fitch'] = hoja_rating_soberano['fitch'].replace({'u':''}, regex=True)

#Separar la hoja homologacion_pais y convertirlo en un dataframe pandas
hoja_homologacion_pais= pd.read_excel('data.xlsx',sheet_name='homologacion_pais')

#Separar la hoja homologacion_rating y convertirlo en un dataframe pandas
hoja_homologacion_rating= pd.read_excel('data.xlsx',sheet_name='homologacion_rating')

#Eliminar el directorio con la carpeta data y crearla vacia
!rm -r /content/data
!mkdir /content/data

# Crear el archivo rating_empresa.csv en la carpeta data
hoja_rating_empresa.to_csv('data/rating_empresa.csv', index=False, encoding='utf-8', header=False)

# Crear el archivo rating_soberano.csv en la carpeta data
hoja_rating_soberano.to_csv('data/rating_soberano.csv', index=False, encoding='utf-8', header=False)

# Crear el archivo homologacion_pais.csv en la carpeta data
hoja_homologacion_pais.to_csv('data/homologacion_pais.csv', index=False, encoding='utf-8', header=False)

# Crear el archivo homologacion_rating.csv en la carpeta data
hoja_homologacion_rating.to_csv('data/homologacion_rating.csv', index=False, encoding='utf-8', header=False)


In [39]:
from pyspark.sql.functions import *

#Leer archivo rating_empresa.csv y convertirlo en dataframe pyspark
df_rating_empresa = spark.read.csv('data/rating_empresa.csv')

#Renombrar los campos de la cabecera (header)
df1_rating_empresa=(df_rating_empresa.withColumnRenamed('_c0','rut')
                                    .withColumnRenamed('_c1','dv')
                                    .withColumnRenamed('_c2','nombre')
                                    .withColumnRenamed('_c3','pais_bbg')
                                    .withColumnRenamed('_c4','mdy')
                                    .withColumnRenamed('_c5','sp')
                                    .withColumnRenamed('_c6','fitch')
                                    .withColumnRenamed('_c7','tipo')
                    )

#Crear nuevos campos 'SP, 'MDY' y 'FITCH'
df1_rating_empresa=(df1_rating_empresa.withColumn('join_mdy',when(col('mdy')!='','MDY'))
                                      .withColumn('join_sp',when(col('sp')!='','SP'))
                                      .withColumn('join_fitch',when(col('fitch')!='','FITCH'))
                   )

#Crear nuevo campo-llave agencia_rating_mdy, agencia_rating_sp, agencia_rating_fitch para cruzarla con el dataframe homologacion_rating
df1_rating_empresa=(df1_rating_empresa.withColumn('agencia_rating_mdy',concat(col('join_mdy'),col('mdy'))).drop('join_mdy')
                                      .withColumn('agencia_rating_sp',concat(col('join_sp'),col('sp'))).drop('join_sp')
                                      .withColumn('agencia_rating_fitch',concat(col('join_fitch'),col('fitch'))).drop('join_fitch')
                   )

df1_rating_empresa.printSchema()
df1_rating_empresa.show(truncate=False)


root
 |-- rut: string (nullable = true)
 |-- dv: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- pais_bbg: string (nullable = true)
 |-- mdy: string (nullable = true)
 |-- sp: string (nullable = true)
 |-- fitch: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- agencia_rating_mdy: string (nullable = true)
 |-- agencia_rating_sp: string (nullable = true)
 |-- agencia_rating_fitch: string (nullable = true)

+--------+---+-------------------------------------------+------------------------+---+----+-----+---------+------------------+-----------------+--------------------+
|rut     |dv |nombre                                     |pais_bbg                |mdy|sp  |fitch|tipo     |agencia_rating_mdy|agencia_rating_sp|agencia_rating_fitch|
+--------+---+-------------------------------------------+------------------------+---+----+-----+---------+------------------+-----------------+--------------------+
|41203332|9  |INDUSTRIAL AND COMERCIAL BANK OF CHINA 

In [40]:
from pyspark.sql.functions import *

#Leer archivo rating_soberano.csv y convertirlo en dataframe pyspark
df_rating_soberano = spark.read.csv('data/rating_soberano.csv')

#Renombrar los campos de la cabecera (header)
df1_rating_soberano=(df_rating_soberano.withColumnRenamed('_c0','pais_bbg')
                                    .withColumnRenamed('_c1','ticker')
                                    .withColumnRenamed('_c2','region')
                                    .withColumnRenamed('_c3','sp')
                                    .withColumnRenamed('_c4','fitch')
                                    .withColumnRenamed('_c5','mdy')
                    )

#Todos los registros del campo 'pais_bbg' covertirlos en mayuscula
df1_rating_soberano=df1_rating_soberano.withColumn('pais_bbg', upper(col('pais_bbg')))

#Crear nuevos campos 'SP, 'MDY' y 'FITCH'
df1_rating_soberano=(df1_rating_soberano.withColumn('join_mdy',when(col('mdy')!='','MDY'))
                                      .withColumn('join_sp',when(col('sp')!='','SP'))
                                      .withColumn('join_fitch',when(col('fitch')!='','FITCH'))
                   )

#Crear nuevo campo-llave agencia_rating_mdy, agencia_rating_sp, agencia_rating_fitch para cruzarla con el dataframe homologacion_rating
df1_rating_soberano=(df1_rating_soberano.withColumn('agencia_rating_mdy',concat(col('join_mdy'),col('mdy'))).drop('join_mdy')
                                      .withColumn('agencia_rating_sp',concat(col('join_sp'),col('sp'))).drop('join_sp')
                                      .withColumn('agencia_rating_fitch',concat(col('join_fitch'),col('fitch'))).drop('join_fitch')
                   )

df1_rating_soberano.printSchema()
df1_rating_soberano.show(truncate=False)

root
 |-- pais_bbg: string (nullable = true)
 |-- ticker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- sp: string (nullable = true)
 |-- fitch: string (nullable = true)
 |-- mdy: string (nullable = true)
 |-- agencia_rating_mdy: string (nullable = true)
 |-- agencia_rating_sp: string (nullable = true)
 |-- agencia_rating_fitch: string (nullable = true)

+---------------------+------------------+---------------------+--------------------+--------------------+--------------------+-----------------------+----------------------+-------------------------+
|pais_bbg             |ticker            |region               |sp                  |fitch               |mdy                 |agencia_rating_mdy     |agencia_rating_sp     |agencia_rating_fitch     |
+---------------------+------------------+---------------------+--------------------+--------------------+--------------------+-----------------------+----------------------+-------------------------+
|ABU DHABI        

In [41]:
#Leer archivo homologacion_rating.csv y convertirlo en dataframe pyspark
df_homologacion_rating = spark.read.csv('data/homologacion_rating.csv')

#Renombrar los campos de la cabecera (header)
df1_homologacion_rating=(df_homologacion_rating.withColumnRenamed('_c0','orden_agencia')
                                    .withColumnRenamed('_c1','agencia')
                                    .withColumnRenamed('_c2','agencia_homol')
                                    .withColumnRenamed('_c3','rating')
                                    .withColumnRenamed('_c4','rating_norma')
                                    .withColumnRenamed('_c5','orden_norma')
                    )

#Crear nuevo campo-llave agencia_homol_rating para cruzarlo con el dataframe rating_empresa y rating_soberano
df1_homologacion_rating=df1_homologacion_rating.withColumn('agencia_homol_rating',concat(col('agencia_homol'),col('rating')))

df1_homologacion_rating.printSchema()
df1_homologacion_rating.show(truncate=False)

root
 |-- orden_agencia: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- agencia_homol: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_norma: string (nullable = true)
 |-- orden_norma: string (nullable = true)
 |-- agencia_homol_rating: string (nullable = true)

+-------------+-------+-------------+------+------------+-----------+--------------------+
|orden_agencia|agencia|agencia_homol|rating|rating_norma|orden_norma|agencia_homol_rating|
+-------------+-------+-------------+------+------------+-----------+--------------------+
|1            |S&P    |SP           |AAA   |AAA         |1          |SPAAA               |
|1            |S&P    |SP           |AA+   |AA+         |2          |SPAA+               |
|1            |S&P    |SP           |AA    |AA          |3          |SPAA                |
|1            |S&P    |SP           |AA-   |AA-         |4          |SPAA-               |
|1            |S&P    |SP           |A+    |A+     

In [42]:
#Leer archivo homologacion_pais.csv y convertirlo en dataframe pyspark
df_homologacion_pais = spark.read.csv('data/homologacion_pais.csv')

#Renombrar los campos de la cabecera (header)
df1_homologacion_pais=(df_homologacion_pais.withColumnRenamed('_c0','pais')
                                    .withColumnRenamed('_c1','cod_pais')
                                    .withColumnRenamed('_c2','pais_bbg')
                    )

#Convertir los registros de los campos 'pais' y 'pais_bbg' en mayuscula
df1_homologacion_pais=(df1_homologacion_pais.withColumn('pais', upper(col('pais')))
                                            .withColumn('pais_bbg', upper(col('pais_bbg')))
                       )

df1_homologacion_pais.printSchema()
df1_homologacion_pais.show(3, truncate=False)

root
 |-- pais: string (nullable = true)
 |-- cod_pais: string (nullable = true)
 |-- pais_bbg: string (nullable = true)

+------------------+--------+---------------------+
|pais              |cod_pais|pais_bbg             |
+------------------+--------+---------------------+
|ABU DHABI         |-1      |ABU DHABI            |
|ARUBA             |-1      |ARUBA                |
|BAILIA DE GUERNSEY|-1      |BAILIWICK OF GUERNSEY|
+------------------+--------+---------------------+
only showing top 3 rows



In [43]:
from pyspark.sql.functions import *
# Utilizar el dataframe df1_rating_empresa y el dataframe df1_homologacion_rating, para obtener los order_nomas o rating

#Cruzar el dataframe df1_rating_empresa con el df1_homologacion_rating para obtener el campo 'orden_norma'(rating) de sp
df_rating_empresa_sp=df1_rating_empresa.join(df1_homologacion_rating, df1_rating_empresa.agencia_rating_sp==df1_homologacion_rating.agencia_homol_rating,'left')
df_rating_empresa_sp=df_rating_empresa_sp.withColumn('orden_norma_sp',when(col('agencia_rating_sp')==col('agencia_homol_rating'),col('orden_norma')))
df_rating_empresa_sp=(df_rating_empresa_sp.withColumnRenamed('rut','rut_sp')
                                    .withColumnRenamed('dv','dv_sp')
                                    .withColumnRenamed('nombre','nombre_sp')
                                    .withColumnRenamed('pais_bbg','pais_bbg_sp')
                                    .withColumnRenamed('tipo','tipo_sp')
                    )

#Seleccionar los campos necesarios de dataframe df_rating_empresa_sp
df_rating_empresa_sp=df_rating_empresa_sp.select('rut_sp','dv_sp','nombre_sp','pais_bbg_sp','sp','tipo_sp','agencia_rating_sp','orden_norma_sp')

#Cruzar el dataframe df1_rating_empresa con el df1_homologacion_rating para obtener el campo 'orden_norma'(rating) de mdy
df_rating_empresa_mdy=df1_rating_empresa.join(df1_homologacion_rating, df1_rating_empresa.agencia_rating_mdy==df1_homologacion_rating.agencia_homol_rating,'left')
df_rating_empresa_mdy=df_rating_empresa_mdy.withColumn('orden_norma_mdy',when(col('agencia_rating_mdy')==col('agencia_homol_rating'),col('orden_norma')))
df_rating_empresa_mdy=(df_rating_empresa_mdy.withColumnRenamed('rut','rut_mdy')
                                    .withColumnRenamed('dv','dv_mdy')
                                    .withColumnRenamed('nombre','nombre_mdy')
                                    .withColumnRenamed('pais_bbg','pais_bbg_mdy')
                                    .withColumnRenamed('tipo','tipo_mdy')
                    )

#Seleccionar los campos necesarios de dataframe df_rating_empresa_mdy
df_rating_empresa_mdy=df_rating_empresa_mdy.select('rut_mdy','dv_mdy','nombre_mdy','pais_bbg_mdy','mdy','tipo_mdy','agencia_rating_mdy','orden_norma_mdy')

#Cruzar el dataframe df1_rating_empresa con el df1_homologacion_rating para obtener el campo 'orden_norma'(rating) de fitch
df_rating_empresa_fitch=df1_rating_empresa.join(df1_homologacion_rating, df1_rating_empresa.agencia_rating_fitch==df1_homologacion_rating.agencia_homol_rating,'left')
df_rating_empresa_fitch=df_rating_empresa_fitch.withColumn('orden_norma_fitch',when(col('agencia_rating_fitch')==col('agencia_homol_rating'),col('orden_norma')))
df_rating_empresa_fitch=(df_rating_empresa_fitch.withColumnRenamed('rut','rut_fitch')
                                    .withColumnRenamed('dv','dv_fitch')
                                    .withColumnRenamed('nombre','nombre_fitch')
                                    .withColumnRenamed('pais_bbg','pais_bbg_fitch')
                                    .withColumnRenamed('tipo','tipo_fitch')
                    )
#Seleccionar los campos necesarios de dataframe df_rating_empresa_fitch
df_rating_empresa_fitch=df_rating_empresa_fitch.select('rut_fitch','dv_fitch','nombre_fitch','pais_bbg_fitch','fitch','tipo_fitch','agencia_rating_fitch','orden_norma_fitch')

df_rating_empresa_sp.show(3,truncate=False)
df_rating_empresa_mdy.show(3,truncate=False)
df_rating_empresa_fitch.show(3,truncate=False)


+--------+-----+--------------------------------------+-----------+---+---------+-----------------+--------------+
|rut_sp  |dv_sp|nombre_sp                             |pais_bbg_sp|sp |tipo_sp  |agencia_rating_sp|orden_norma_sp|
+--------+-----+--------------------------------------+-----------+---+---------+-----------------+--------------+
|41203332|9    |INDUSTRIAL AND COMERCIAL BANK OF CHINA|CHINA      |A  |Solvencia|SPA              |6             |
|40000177|4    |ITAU UNIBANCO S-A                     |BRAZIL     |BB-|Solvencia|SPBB-            |13            |
|41001123|9    |BCO BRADESCO SA BRASIL                |BRAZIL     |BB-|Solvencia|SPBB-            |13            |
+--------+-----+--------------------------------------+-----------+---+---------+-----------------+--------------+
only showing top 3 rows

+--------+------+--------------------------------------+------------+---+---------+------------------+---------------+
|rut_mdy |dv_mdy|nombre_mdy                        

In [44]:
from pyspark.sql.functions import *
#Unir por rut empresa los rating Empresa de sp, mdy,fitch y y obtener el mayor rating

#Unir el dataframe df_rating_empresa_sp, df_rating_empresa_mdy y df_rating_empresa_fitch
df_rating_empresa_cons =df_rating_empresa_sp.join(df_rating_empresa_mdy, df_rating_empresa_sp.rut_sp==df_rating_empresa_mdy.rut_mdy,'inner' )
df_rating_empresa_cons=df_rating_empresa_cons.select('rut_sp','dv_sp','nombre_sp','pais_bbg_sp','sp','orden_norma_sp','mdy','orden_norma_mdy')

df_rating_empresa_cons=df_rating_empresa_cons.join(df_rating_empresa_fitch, df_rating_empresa_cons.rut_sp==df_rating_empresa_fitch.rut_fitch,'inner' )
df_rating_empresa_cons=df_rating_empresa_cons.select('rut_sp','dv_sp','nombre_sp','pais_bbg_sp','sp','orden_norma_sp','mdy','orden_norma_mdy','fitch','orden_norma_fitch')
df_rating_empresa_cons=(df_rating_empresa_cons.withColumnRenamed('rut_sp','rut')
                                    .withColumnRenamed('dv_sp','dv')
                                    .withColumnRenamed('nombre_sp','nombre')
                                    .withColumnRenamed('pais_bbg_sp','pais_bbg')
                    )

#Convertir a numero (int) los campos rating 'orden_norma_sp','orden_norma_mdy' y 'orden_norma_fitch' del dataframe df_rating_empresa_cons
df_rating_empresa_cons=(df_rating_empresa_cons.withColumn('orden_norma_sp',df_rating_empresa_cons.orden_norma_sp.cast('int'))
                                                .withColumn('orden_norma_mdy',df_rating_empresa_cons.orden_norma_mdy.cast('int'))
                                                .withColumn('orden_norma_fitch',df_rating_empresa_cons.orden_norma_fitch.cast('int'))
                        )

#Obtener el mayor rating de 'orden_norma_sp','orden_norma_mdy' y 'orden_norma_fitch' del dataframe df_rating_empresa_cons
#Suposicion para los casos que el orden_norma es nulo, se mantendran de esa forma, como lo mencionan en el test (los campos que no cruzan con la tabla de homologacion mantener nulos o blancos)
df_rating_empresa_cons=df_rating_empresa_cons.withColumn('rating_empresa',when((col('orden_norma_sp')>=col('orden_norma_mdy')) & (col('orden_norma_sp')>=col('orden_norma_fitch')),col('sp')).otherwise('null'))
df_rating_empresa_cons=df_rating_empresa_cons.withColumn('rating_empresa',when((col('orden_norma_mdy')>=col('orden_norma_sp')) & (col('orden_norma_mdy')>=col('orden_norma_fitch')),col('mdy')).otherwise(col('rating_empresa')))
df_rating_empresa_cons=df_rating_empresa_cons.withColumn('rating_empresa',when((col('orden_norma_fitch')>=col('orden_norma_sp')) & (col('orden_norma_fitch')>=col('orden_norma_mdy')),col('fitch')).otherwise(col('rating_empresa')))

#Cruzar el dataframe df_rating_empresa_cons con el df1_homologacion_pais, para obtener el pais homologado
df_rating_empresa_cons=df_rating_empresa_cons.join(df1_homologacion_pais, df_rating_empresa_cons.pais_bbg==df1_homologacion_pais.pais_bbg,'left')

#Seleccionar los campos necesarios de dataframe df_rating_empresa_cons
df_rating_empresa_cons=df_rating_empresa_cons.select('rut','dv','nombre','pais','rating_empresa')

df_rating_empresa_cons.show(20,truncate=False)


+--------+---+-------------------------------------------+--------------+--------------+
|rut     |dv |nombre                                     |pais          |rating_empresa|
+--------+---+-------------------------------------------+--------------+--------------+
|41203332|9  |INDUSTRIAL AND COMERCIAL BANK OF CHINA     |CHINA         |A             |
|40000177|4  |ITAU UNIBANCO S-A                          |BRASIL        |BB-           |
|41001123|9  |BCO BRADESCO SA BRASIL                     |BRASIL        |BB-           |
|42001513|5  |BANCO SANTANDER BRASIL                     |BRASIL        |null          |
|41001310|K  |BCO DO BRASIL S A-BRASIL                   |BRASIL        |BB-           |
|41202256|4  |BANK OF CHINA HONG KONG BRA - CHINA        |CHINA         |A             |
|44000061|4  |CHINA CONSTRUCTION BANK                    |CHINA         |A             |
|41001391|6  |CITIBANK NA ESTADOS UNIDOS                 |ESTADOS UNIDOS|A+            |
|41001363|0  |BCO SAF

In [45]:
from pyspark.sql.functions import *
# Utilizar el dataframe df1_rating_soberano y el dataframe df1_homologacion_rating, para obtener los order_nomas o rating

#Cruzar el dataframe df1_rating_soberano con el df1_homologacion_rating para obtener el campo 'orden_norma'(rating) de sp
df_rating_soberano_sp=df1_rating_soberano.join(df1_homologacion_rating, df1_rating_soberano.agencia_rating_sp==df1_homologacion_rating.agencia_homol_rating,'left')
df_rating_soberano_sp=df_rating_soberano_sp.withColumn('orden_norma_sp',when(col('agencia_rating_sp')==col('agencia_homol_rating'),col('orden_norma')))
df_rating_soberano_sp=(df_rating_soberano_sp.withColumnRenamed('pais_bbg','pais_bbg_sp')
                                    .withColumnRenamed('ticker','ticker_sp')
                                    .withColumnRenamed('region','region_sp')
                    )

#Seleccionar los campos necesarios de dataframe df_rating_soberano_sp
df_rating_soberano_sp=df_rating_soberano_sp.select('pais_bbg_sp','ticker_sp','region_sp','sp','agencia_rating_sp','orden_norma_sp')


#Cruzar el dataframe df1_rating_soberano con el df1_homologacion_rating para obtener el campo 'orden_norma'(rating) de mdy
df_rating_soberano_mdy=df1_rating_soberano.join(df1_homologacion_rating, df1_rating_soberano.agencia_rating_mdy==df1_homologacion_rating.agencia_homol_rating,'left')
df_rating_soberano_mdy=df_rating_soberano_mdy.withColumn('orden_norma_mdy',when(col('agencia_rating_mdy')==col('agencia_homol_rating'),col('orden_norma')))
df_rating_soberano_mdy=(df_rating_soberano_mdy.withColumnRenamed('pais_bbg','pais_bbg_mdy')
                                    .withColumnRenamed('ticker','ticker_mdy')
                                    .withColumnRenamed('region','region_mdy')
                    )

#Seleccionar los campos necesarios de dataframe df_rating_soberano_mdy
df_rating_soberano_mdy=df_rating_soberano_mdy.select('pais_bbg_mdy','ticker_mdy','region_mdy','mdy','agencia_rating_mdy','orden_norma_mdy')

#Cruzar el dataframe df1_rating_soberano con el df1_homologacion_rating para obtener el campo 'orden_norma'(rating) de fitch
df_rating_soberano_fitch=df1_rating_soberano.join(df1_homologacion_rating, df1_rating_soberano.agencia_rating_fitch==df1_homologacion_rating.agencia_homol_rating,'left')
df_rating_soberano_fitch=df_rating_soberano_fitch.withColumn('orden_norma_fitch',when(col('agencia_rating_fitch')==col('agencia_homol_rating'),col('orden_norma')))
df_rating_soberano_fitch=(df_rating_soberano_fitch.withColumnRenamed('pais_bbg','pais_bbg_fitch')
                                    .withColumnRenamed('ticker','ticker_fitch')
                                    .withColumnRenamed('region','region_fitch')
                    )

#Seleccionar los campos necesarios de dataframe df_rating_soberano_fitch
df_rating_soberano_fitch=df_rating_soberano_fitch.select('pais_bbg_fitch','ticker_fitch','region_fitch','fitch','agencia_rating_fitch','orden_norma_fitch')

df_rating_soberano_sp.show(3,truncate=False)
df_rating_soberano_mdy.show(3,truncate=False)
df_rating_soberano_fitch.show(3,truncate=False)


+-----------+------------------+------------------+----+-----------------+--------------+
|pais_bbg_sp|ticker_sp         |region_sp         |sp  |agencia_rating_sp|orden_norma_sp|
+-----------+------------------+------------------+----+-----------------+--------------+
|ABU DHABI  |1022337Z UH Equity|Africa/Middle East|AA  |SPAA             |3             |
|ALBANIA    |1001Z AL Equity   |Eastern Europe    |B+  |SPB+             |14            |
|ALGERIA    |3700Z US Equity   |Africa/Middle East|null|null             |null          |
+-----------+------------------+------------------+----+-----------------+--------------+
only showing top 3 rows

+------------+------------------+------------------+----+------------------+---------------+
|pais_bbg_mdy|ticker_mdy        |region_mdy        |mdy |agencia_rating_mdy|orden_norma_mdy|
+------------+------------------+------------------+----+------------------+---------------+
|ABU DHABI   |1022337Z UH Equity|Africa/Middle East|Aa2 |MDYAa2   

In [46]:
from pyspark.sql.functions import *
#Unir por pais los rating soberano de sp, mdy,fitch y obtener el mayor rating

#Unir el dataframe df_rating_soberano_sp, df_rating_soberano_mdy y df_rating_soberano_fitch
df_rating_soberano_cons=df_rating_soberano_sp.join(df_rating_soberano_mdy, df_rating_soberano_sp.pais_bbg_sp==df_rating_soberano_mdy.pais_bbg_mdy,'inner' ) 
df_rating_soberano_cons=df_rating_soberano_cons.select('pais_bbg_sp','ticker_sp','region_sp','sp','orden_norma_sp','mdy','orden_norma_mdy')

df_rating_soberano_cons=df_rating_soberano_cons.join(df_rating_soberano_fitch, df_rating_soberano_cons.pais_bbg_sp==df_rating_soberano_fitch.pais_bbg_fitch,'inner' )
df_rating_soberano_cons=df_rating_soberano_cons.select('pais_bbg_sp','ticker_sp','region_sp','sp','orden_norma_sp','mdy','orden_norma_mdy','fitch','orden_norma_fitch')
df_rating_soberano_cons=(df_rating_soberano_cons.withColumnRenamed('pais_bbg_sp','pais_bbg')
                                    .withColumnRenamed('ticker_sp','ticker')
                                    .withColumnRenamed('region_sp','region')
                    )

#Convertir a numero (int) los campos rating 'orden_norma_sp','orden_norma_mdy' y 'orden_norma_fitch' del dataframe df_rating_empresa_cons
df_rating_soberano_cons=(df_rating_soberano_cons.withColumn('orden_norma_sp',df_rating_soberano_cons.orden_norma_sp.cast('int'))
                                                .withColumn('orden_norma_mdy',df_rating_soberano_cons.orden_norma_mdy.cast('int'))
                                                .withColumn('orden_norma_fitch',df_rating_soberano_cons.orden_norma_fitch.cast('int'))
                        )

#Obtener el mayor rating de 'orden_norma_sp','orden_norma_mdy' y 'orden_norma_fitch' del dataframe df_rating_empresa_cons
#Suposicion para los casos que el orden_norma es nulo, se mantendran de esa forma, como lo mencionan en el test (los campos que no cruzan con la tabla de homologacion mantener nulos o blancos)
df_rating_soberano_cons=df_rating_soberano_cons.withColumn('rating_soberano',when((col('orden_norma_sp')>=col('orden_norma_mdy')) & (col('orden_norma_sp')>=col('orden_norma_fitch')),col('sp')).otherwise('null'))
df_rating_soberano_cons=df_rating_soberano_cons.withColumn('rating_soberano',when((col('orden_norma_mdy')>=col('orden_norma_sp')) & (col('orden_norma_mdy')>=col('orden_norma_fitch')),col('mdy')).otherwise(col('rating_soberano')))
df_rating_soberano_cons=df_rating_soberano_cons.withColumn('rating_soberano',when((col('orden_norma_fitch')>=col('orden_norma_sp')) & (col('orden_norma_fitch')>=col('orden_norma_mdy')),col('fitch')).otherwise(col('rating_soberano')))

#Cruzar el dataframe df_rating_empresa_cons con el df1_homologacion_pais, para obtener el pais homologado
df_rating_soberano_cons=df_rating_soberano_cons.join(df1_homologacion_pais, df_rating_soberano_cons.pais_bbg==df1_homologacion_pais.pais_bbg,'left')
df_rating_soberano_cons=df_rating_soberano_cons.withColumnRenamed('pais','pais_h')

#Seleccionar los campos necesarios de dataframe df_rating_soberano_cons
df_rating_soberano_cons=df_rating_soberano_cons.select('pais_h','rating_soberano')

df_rating_soberano_cons.show(3,truncate=False)



+---------+---------------+
|pais_h   |rating_soberano|
+---------+---------------+
|ABU DHABI|AA             |
|ALBANIA  |null           |
|ARGELIA  |null           |
+---------+---------------+
only showing top 3 rows



In [47]:
from pyspark.sql.functions import *
#UNIR RATING EMPRESA Y RATING SOBERANO POR EMPRESA

df_maestro_rating=df_rating_empresa_cons.join(df_rating_soberano_cons, df_rating_empresa_cons.pais==df_rating_soberano_cons.pais_h,'left')
df_maestro_rating=df_maestro_rating.select('rut','dv','nombre','pais','rating_empresa','rating_soberano')
df_maestro_rating.show(20,truncate=False)

#Eliminar el directorio con la carpeta ouput
!rm -r /content/ouput

#Crear carpeta output y archivo csv
df_maestro_rating.write.options(header='True', delimiter='|').csv('ouput/')

#Renombrar el archivo.csv
!mv /content/ouput/*.csv /content/ouput/maestro_rating.csv




+--------+---+-------------------------------------------+--------------+--------------+---------------+
|rut     |dv |nombre                                     |pais          |rating_empresa|rating_soberano|
+--------+---+-------------------------------------------+--------------+--------------+---------------+
|41203226|8  |KOOKMIN BANK - COREA                       |COREA DEL SUR |A             |AA-            |
|40000154|5  |BNP PARIBAS S.A. FRANCIA                   |FRANCIA       |A+            |AA             |
|41203332|9  |INDUSTRIAL AND COMERCIAL BANK OF CHINA     |CHINA         |A             |A+             |
|41202256|4  |BANK OF CHINA HONG KONG BRA - CHINA        |CHINA         |A             |A+             |
|44000061|4  |CHINA CONSTRUCTION BANK                    |CHINA         |A             |A+             |
|41202507|5  |AGRICULTURAL BANK OF CHINA THE-CHINA       |CHINA         |A             |A+             |
|41001007|0  |Mizuho Bank Ltd.                         